# Run Parameters
### Edit this section to configure your run. 
You don't need to edit any of the other sections to run the simulation, unless you want to 
change the algorithm or the visualizations.
#####
image_path: The relative or absolute path to the image you would like to process. Please note it has to be square 
(height=width) (path: string)

Resolution reduction parameters:

cutoff_frequency: Max sampling frequency to simulate in cycles per image (number: integer)

tukey_alpha: The tapering factor for the Tukey filter (number: float)

tukey_radius_expansion_factor: Additional apodization factor for the tukey filter (number: float)

Contrast reduction parameters:

gamma_lst: A list of Gamma values to simulate the Gamma contrast reduction option (list)

gain_lst: A list of Gain values to simulate the Sigmoid contrast reduction option (list)

sigmoid_x_shift: The horizontal shift to simulate in the sigmoid contrast reduction (number: float)

In [ ]:
image_path = "sample_image.jpg"

# resolution reduction parameters
cutoff_frequency = 10 
tukey_alpha = 0.3 
tukey_radius_expansion_factor = 0.3 

# contrast reduction parameters
gamma_lst = [1.7, 2.6, 3.5] 
gain_lst = [10, 20, 30] 
sigmoid_x_shift = 0.2 

# Imports

In [ ]:
# standard imports
import sys
import shutil
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Rectangle

# install mediapipe
!pip install mediapipe > /dev/null 2>&1

# mediapipe imports
from mediapipe.framework.formats import landmark_pb2
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe.python.solutions.face_mesh_connections import FACEMESH_LEFT_IRIS, FACEMESH_RIGHT_IRIS

# project imports
from utils import *
from image_processing import *
from plot import *
from facial_landmarking import *

# download face landmarker task file
if shutil.which("wget") is None:  
    print("wget not found. Installing...")

    if sys.platform.startswith("darwin"):  
        print("Detected macOS. Installing wget using Homebrew...")
        !brew install wget

    elif sys.platform.startswith("win"):  
        print("Detected Windows. Installing wget using Chocolatey or Winget...")
        try:
            !choco install wget -y  
        except:
            print("Chocolatey not found. Trying Winget...")
            !winget install --id GnuWin32.Wget  

    elif sys.platform.startswith("linux"):  
        print("Detected Linux. Installing wget using package manager...")
        try:
            !sudo apt update && sudo apt install -y wget  
        except:
            print("APT not found. Trying Yum...")
            !sudo yum install -y wget  

    else:
        print("Unsupported OS. Please install wget manually.")

!wget -O face_landmarker_v2_with_blendshapes.task -q https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/1/face_landmarker.task



# Simulate prosthetic vision

## Simulate loss of spatial resolution 

### Load in an image, convert to grayscale and create a tukey window

In [ ]:
gray_img = rgb_to_gray(image_path)
img_size = gray_img.shape[0]
tukey_win = create_tukey_window(cutoff_frequency, img_size)

### Plot tukey window in 1D

In [ ]:
fig=plt.figure(figsize=(7,3))

midcoord = img_size // 2 - 1

# Plot the filter
plt.plot(tukey_win[midcoord, midcoord - (2 * cutoff_frequency) : midcoord + (2 * cutoff_frequency) + 1], c='black')

# Add horizontal and vertical reference lines
plt.axvline(x=2 * cutoff_frequency, color='r', linestyle='--', linewidth=0.5)
plt.axvline(x=cutoff_frequency, color='r', linestyle='--', linewidth=0.5)
plt.axvline(x=3 * cutoff_frequency, color='r', linestyle='--', linewidth=0.5)
plt.axvline(x=2 * cutoff_frequency + cutoff_frequency*(tukey_alpha + 1), color='r', linestyle='--', linewidth=0.5)
plt.axvline(x=2 * cutoff_frequency - cutoff_frequency*(tukey_alpha + 1), color='r', linestyle='--', linewidth=0.5)

# Set x-ticks at each integer
plt.xticks(range(0, 4 * cutoff_frequency + 1))

# Get current x-tick positions and set labels every 5th tick
current_ticks = plt.gca().get_xticks()  # Get current x-tick positions (e.g., from 0 to 40)
new_labels = [tick - 2 * cutoff_frequency if tick % 5 == 0 else "" for tick in current_ticks]  # Label every 5th tick, others are empty

# Apply the new labels and maintain the tick positions
plt.xticks(current_ticks, new_labels)
plt.xlabel('Cycles per image')
plt.ylabel('Amplitude')
plt.ylim([0.0, 1.05])

# Limit the plot range to show the desired x-axis range
plt.xlim([0, 4 * cutoff_frequency])

plt.show()

### Plot input image in the frequency domain

In [ ]:
magnitude_spectrum_log = fft_plot(gray_img)

freq_y = np.fft.fftshift(np.fft.fftfreq(img_size)) * img_size # Frequency axis for rows
freq_x = np.fft.fftshift(np.fft.fftfreq(img_size)) * img_size # Frequency axis for columns

plt.figure(figsize=(3, 3))
plt.imshow(magnitude_spectrum_log, extent=[freq_x[0], freq_x[-1], freq_y[0], freq_y[-1]], cmap='gray')
plt.xlabel("Cycles per image")
plt.ylabel("Cycles per image")
plt.show()

### Plot the custom filter in 2D (zoomed in)

In [ ]:
zoom_into_cycles_num = 20
plt.figure(figsize=(3, 3))
ax = plt.gca()

# Plot the image
im = ax.imshow(tukey_win, extent=[freq_x[0], freq_x[-1], freq_y[0], freq_y[-1]])
plt.xlim([-zoom_into_cycles_num, zoom_into_cycles_num])
plt.ylim([-zoom_into_cycles_num, zoom_into_cycles_num])
plt.xlabel("Cycles per image")
plt.ylabel("Cycles per image")
yticks = np.linspace(-zoom_into_cycles_num, zoom_into_cycles_num, 5)
plt.yticks(yticks)

# Create a colorbar that matches the height of the image
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
cbar = plt.colorbar(im, cax=cax)
cbar.set_label("Amplitude")

plt.show()

## Simulate loss of contrast

### Draw tone curve functions for gamma and sigmoid transforms

####Gamma Transform

In [ ]:
# Generate x values between 0 and 1
x = np.linspace(0, 1, 100)
linear_y = linear_function(x)
colors = plt.cm.Reds(np.linspace(0.4, 1, len(gamma_lst)))  # 0.4 to 1 for darker shades

# Plot the function
fig, ax = plt.subplots(figsize=(5, 5))
plt.plot(x, linear_y, linestyle=':', linewidth=1)
for gamma, color in zip(gamma_lst, colors):
    plt.plot(x, [gamma_transform(x_val, gamma) for x_val in x],
             linestyle='-', linewidth=2, label=fr"$\gamma = {gamma}$", color=color)
plt.xlim(0, 1)
plt.ylim(0, 1)
ax.set_xticks([])
ax.set_yticks([])
plt.legend()

# Create grayscale colormap
grayscale_cmap = LinearSegmentedColormap.from_list("grayscale", ["black", "white"])

# X-axis color bar overlay
cbar_ax_x = fig.add_axes([ax.get_position().x0, ax.get_position().y0 - 0.05,
                          ax.get_position().width, 0.03])
cbar_x = fig.colorbar(plt.cm.ScalarMappable(cmap=grayscale_cmap), cax=cbar_ax_x, orientation='horizontal')
cbar_x.set_label("Input Grayscale", labelpad=0)

# Y-axis color bar overlay
cbar_ax_y = fig.add_axes([ax.get_position().x0 - 0.05, ax.get_position().y0,
                          0.03, ax.get_position().height])
cbar_y = fig.colorbar(plt.cm.ScalarMappable(cmap=grayscale_cmap), cax=cbar_ax_y, orientation='vertical')
cbar_y.set_label("Output Grayscale", labelpad=0)
cbar_y.ax.yaxis.set_ticks_position('left')  # Position ticks on the left
cbar_y.ax.yaxis.set_label_position('left')  # Position label on the left

plt.show()


#### Sigmoid Transform

In [ ]:
# Generate x values between 0 and 1
x = np.linspace(0, 1, 100)
linear_y = linear_function(x)

# Create a colormap
colors = plt.cm.Blues(np.linspace(0.4, 1, len(gamma_lst)))

# Plot the function
fig, ax = plt.subplots(figsize=(5, 5))
plt.plot(x, linear_y, linestyle=':', linewidth=1)
for s, color in zip(gain_lst, colors):
    plt.plot(x, [sigmoid_transform(x_val, s, sigmoid_x_shift) for x_val in x],
             linestyle='-', linewidth=2, label=fr"$\mathrm{{g}} = {s}$", color=color)
plt.xlim(0, 1)
plt.ylim(0, 1)
ax.set_xticks([])
ax.set_yticks([])
plt.legend()

# Create grayscale colormap
grayscale_cmap = LinearSegmentedColormap.from_list("grayscale", ["black", "white"])

# X-axis color bar overlay
cbar_ax_x = fig.add_axes([ax.get_position().x0, ax.get_position().y0 - 0.05,
                          ax.get_position().width, 0.03])
cbar_x = fig.colorbar(plt.cm.ScalarMappable(cmap=grayscale_cmap), cax=cbar_ax_x, orientation='horizontal')
cbar_x.set_label("Input Grayscale", labelpad=0)

# Y-axis color bar overlay
cbar_ax_y = fig.add_axes([ax.get_position().x0 - 0.05, ax.get_position().y0,
                          0.03, ax.get_position().height])
cbar_y = fig.colorbar(plt.cm.ScalarMappable(cmap=grayscale_cmap), cax=cbar_ax_y, orientation='vertical')
cbar_y.set_label("Output Grayscale", labelpad=0)
cbar_y.ax.yaxis.set_ticks_position('left')
cbar_y.ax.yaxis.set_label_position('left')

plt.show()


### Draw a gradient bar to visualize the effect of the transforms

In [ ]:
thickness = 20

# Create a gradient array that goes from 0 (black) to 1 (white)
gradient = np.linspace(0, 1, 256)  # 256 shades from black to white
gradient = np.tile(gradient, (thickness, 1))  # Repeat the gradient as a bar

gamma_transformed = [gamma_transform(each_grayscale, gamma_lst[-1]) for each_grayscale in gradient[0]]
sigmoid_transformed = [sigmoid_transform(each_grayscale, gain_lst[-1], sigmoid_x_shift) for each_grayscale in gradient[0]]

fig = plt.figure(figsize=(5, 2))

# Add the first subplot (top)
plt.subplot(2, 1, 1)  # 1 row, 2 columns, first subplot

# Plot the sigmoid-transformed data on the first subplot
plt.imshow(np.tile(sigmoid_transformed, (thickness, 1)), aspect='auto', cmap='gray')
plt.axis('off')

# Add a black border around the first subplot
rect = Rectangle(
    (0, 0),  # Starting point
    1, 1,  # Width and height in relative coordinates
    transform=plt.gca().transAxes,  # Use axes-relative coordinates
    linewidth=2, edgecolor='black', facecolor='none'
)
plt.gca().add_patch(rect)

# Set title for the first subplot
plt.title(f'g = {gain_lst[-1]} transformed grayscale')#, fontsize=15, pad=10)

# Add the second subplot (bottom)
plt.subplot(2, 1, 2)  # 1 row, 2 columns, second subplot

# Plot the gamma-transformed data on the second subplot
plt.imshow(np.tile(gamma_transformed, (thickness, 1)), aspect='auto', cmap='gray')
plt.axis('off')

# Add a black border around the second subplot
rect2 = Rectangle(
    (0, 0),  # Starting point
    1, 1,  # Width and height in relative coordinates
    transform=plt.gca().transAxes,  # Use axes-relative coordinates
    linewidth=2, edgecolor='black', facecolor='none'
)
plt.gca().add_patch(rect2)

# Set title for the second subplot
plt.title(f'$\gamma =$ {gamma_lst[-1]} transformed grayscale') #, fontsize=15, pad=10)

# Show the plot
plt.tight_layout()  # Adjust layout to prevent overlap
plt.show()

## Simulate a face

In [ ]:
face_img = rgb_to_gray(image_path)
face_tukey_win = create_tukey_window(cutoff_frequency, gray_img.shape[0])
spatially_filtered_face_img = fft_filter(face_img, face_tukey_win)

In [ ]:
face_img_lst = [face_img, spatially_filtered_face_img]

Simulate a face using a Gamma transform

In [ ]:
# create new figure
i = 1
plt.figure(figsize=(13, 7))
plt.suptitle('Gamma Transform', fontsize=16)

# Loop through images and transformations to plot
for img in face_img_lst:
    for gamma in gamma_lst:
        ax = plt.subplot(len(face_img_lst), len(gamma_lst), i)
        validate_gray_img(img)
        transformed_float_img = gamma_transform(img, gamma)
        ax.imshow(transformed_float_img, cmap='gray')
        ax.axis('off')
        plt.title('$\gamma =$ ' + str(gamma))
        i += 1

Simulate a face using a Sigmoid transform

In [ ]:
# create new figure
i = 1
plt.figure(figsize=(13, 7))
plt.suptitle('Sigmoid Transform', fontsize=16)

# Loop through images and transformations to plot
for img in face_img_lst:
    for gain in gain_lst:
        ax = plt.subplot(len(face_img_lst), len(gain_lst), i)
        validate_gray_img(img)
        transformed_float_img = sigmoid_transform(img, gain, sigmoid_x_shift)
        ax.imshow(transformed_float_img, cmap='gray')
        ax.axis('off')
        plt.title('$g =$ ' + str(gain))
        i += 1

## Simulate a Campbell-Robson Chart

In [ ]:
# create a CampbellRobsonChart object
chart_generator = CampbellRobsonChart()

# Draw a gray square CRC with min value 0 and max value 1
gray_crc = chart_generator.chart_data

# Create a new tukey window to fit the dimension of gray_crc
crc_tukey_win = create_tukey_window(cutoff_frequency, gray_crc.shape[0])

# Spatially filter the gray_crc
spatially_filtered_crc = fft_filter(gray_crc, crc_tukey_win)

In [ ]:
crc_img_lst = [gray_crc, spatially_filtered_crc]

Simulate a Campbell-Robson Chart using a Gamma transform

In [ ]:
# initialize variables
i = 1
axes_label = True
ticks = chart_generator.ticks

# create a new figure
plt.figure(figsize=(13, 7))
plt.suptitle('Gamma Transform', fontsize=16)

# Loop through images and transformations
for img in crc_img_lst:
    for gamma in gamma_lst:
        ax = plt.subplot(len(crc_img_lst), len(gamma_lst), i)
        validate_gray_img(img)
        transformed_float_img = gamma_transform(img, gamma)

        ax.imshow(transformed_float_img, cmap='gray', origin='lower')

        # Set x-ticks for frequency labels
        ax.set_xticks(ticks['x_positions'])
        ax.set_xticklabels(ticks['x_labels'])
        if axes_label:
          ax.set_xlabel('Spatial Frequency (cpd)')

        # Set y-ticks for frequency labels
        ax.set_yticks(ticks['y_positions'])
        ax.set_yticklabels(ticks['y_labels'])
        if axes_label:
          ax.set_ylabel('Contrast')
        i += 1

In [ ]:
# To save out individual charts as own png instead of a panel
save_out_individual_pngs = False
if save_out_individual_pngs:
  for img_index, img in enumerate(crc_img_lst, start=1):
      # Save the original image
      chart_generator.save_campbell_robson_chart_png(f'crc_{img_index}_original.png', data=img, axes_label=False)

      # Save gamma-adjusted images
      for gamma in gamma_lst:
          validate_gray_img(img)
          transformed_float_img = gamma_transform(img, gamma)
          chart_generator.save_campbell_robson_chart_png(f'crc_{img_index}_gamma_{gamma}.png', data=transformed_float_img, axes_label=False)

Simulate a Campbell-Robson Chart using a Sigmoid transform


In [ ]:
# initialize variables
i = 1
axes_label = True
ticks = chart_generator.ticks

# create a new figure
plt.figure(figsize=(13, 7))
plt.suptitle('Sigmoid Transform', fontsize=16)

# Loop through images and transformations
for img in crc_img_lst:
    for gain in gain_lst:
        ax = plt.subplot(len(crc_img_lst), len(gain_lst), i)
        validate_gray_img(img)
        transformed_float_img = sigmoid_transform(img, gain, sigmoid_x_shift)

        ax.imshow(transformed_float_img, cmap='gray', origin='lower')

        # Set x-ticks for frequency labels
        ax.set_xticks(ticks['x_positions'])
        ax.set_xticklabels(ticks['x_labels'])
        if axes_label:
          ax.set_xlabel('Spatial Frequency (cpd)')

        # Set y-ticks for frequency labels
        ax.set_yticks(ticks['y_positions'])
        ax.set_yticklabels(ticks['y_labels'])
        if axes_label:
          ax.set_ylabel('Contrast')
        i += 1

In [ ]:
# To save out individual charts as own png instead of a panel
save_out_individual_pngs = False
if save_out_individual_pngs:
  for img_index, img in enumerate(crc_img_lst, start=1):
      # Save the original image
      chart_generator.save_campbell_robson_chart_png(f'crc_{img_index}_original.png', data=img, axes_label=False)

      # Save sigmoid-adjusted images
      for gain in gain_lst:
          validate_gray_img(img)
          transformed_float_img = sigmoid_transform(img, gain, sigmoid_x_shift)
          chart_generator.save_campbell_robson_chart_png(f'crc_{img_index}_sigmoid_{gain}.png', data=transformed_float_img, axes_label=False)



## Simulate a Landolt C

In [ ]:
# draw a landolt C panel
landolt_c_panel = draw_landolt_c_panel()

# create a new tukey window to fit the dimension of landolt_c_panel
landolt_c_panel_tukey_win = create_tukey_window(cutoff_frequency, landolt_c_panel.shape[0])

# spatially filter the landolt_c_panel
spatially_filtered_landolt_c_panel = fft_filter(landolt_c_panel, landolt_c_panel_tukey_win)

In [ ]:
landolt_c_img_lst = [landolt_c_panel, spatially_filtered_landolt_c_panel]

Simulate a Landolt C using a Gamma transform

In [ ]:
# @title
i = 1
plt.figure(figsize=(13, 7))
plt.suptitle('Gamma Transform')
# Loop through images and transformations
for img in landolt_c_img_lst:
    for gamma in gamma_lst:
        ax = plt.subplot(len(landolt_c_img_lst), len(gamma_lst), i)
        validate_gray_img(img)
        transformed_float_img = gamma_transform(img, gamma)
        ax.imshow(transformed_float_img, cmap='gray')
        ax.axis('off')
        plt.title('$\gamma =$ ' + str(gamma))
        i += 1

Simulate a Landolt C using a Sigmoid transform

In [ ]:
# create new figure
i = 1
plt.figure(figsize=(13, 7))
plt.suptitle('Sigmoid Transform')

# Loop through images and transformations
for img in landolt_c_img_lst:
    for gain in gain_lst:
        ax = plt.subplot(len(landolt_c_img_lst), len(gain_lst), i)
        validate_gray_img(img)
        transformed_float_img = sigmoid_transform(img, gain, sigmoid_x_shift)
        ax.imshow(transformed_float_img, cmap='gray')
        ax.axis('off')
        plt.title('$g =$ ' + str(gain))
        i += 1

# Improve prosthetic vision

## Inverse tone curves

In [ ]:
# load the image and run the resolution reduction part
face_img = rgb_to_gray(image_path)
face_tukey_win = create_tukey_window(cutoff_frequency, gray_img.shape[0])
spatially_filtered_face_img = fft_filter(face_img, face_tukey_win)

### Inverse of Gamma (just 1/gamma)

In [ ]:
# contrast reduction with and without preemptive inverse Gamma transform
plt.figure(figsize=(7,7))
plt.subplot(2,2,1)
plt.imshow(face_img, cmap='gray')
plt.title('No inverse transform')
plt.axis('off')

plt.subplot(2,2,2)
inv_gamma_face_img = gamma_transform(face_img, 1 / 3.5)
plt.imshow(inv_gamma_face_img, cmap='gray')
plt.title('Inverse transform')
plt.axis('off')

plt.subplot(2,2,3)
plt.imshow(gamma_transform(spatially_filtered_face_img, 3.5), cmap='gray')
plt.title('PV without inverse transform')
plt.axis('off')

plt.subplot(2,2,4)
spatially_filtered_inverse_img = fft_filter(quantize_image(inv_gamma_face_img), face_tukey_win)
plt.imshow(gamma_transform(spatially_filtered_inverse_img, 3.5), cmap='gray')
plt.title('PV with inverse transform')
plt.axis('off')

Inverse of Sigmoid

In [ ]:
# contrast reduction with and without preemptive inverse Gamma transform
plt.figure(figsize=(7,7))
plt.subplot(2,2,1)
plt.imshow(face_img, cmap='gray')
plt.title('No inverse transform')
plt.axis('off')

plt.subplot(2,2,2)
inv_sigmoid_face_img = inverse_sigmoid(face_img, 30, 0.2)
plt.imshow(inv_sigmoid_face_img, cmap='gray')
plt.title('Inverse transform')
plt.axis('off')

plt.subplot(2,2,3)
plt.imshow(sigmoid_transform(spatially_filtered_face_img, 30, 0.2), cmap='gray')
plt.title('PV without inverse transform')
plt.axis('off')

plt.subplot(2,2,4)
clipped_inv_sigmoid_face_img = np.clip(inv_sigmoid_face_img, 0, 1)
spatially_filtered_inverse_img = fft_filter(quantize_image(clipped_inv_sigmoid_face_img), face_tukey_win)
plt.imshow(sigmoid_transform(spatially_filtered_inverse_img, 30, 0.2), cmap='gray')
plt.title('PV with inverse transform')
plt.axis('off')

## Facial Landmarking

In [ ]:
# Create an FaceLandmarker object.
base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=False,
                                       output_facial_transformation_matrixes=False,
                                       num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)

In [ ]:
# Load in image
image = mp.Image.create_from_file(image_path)

# Landmark detection
detection_result = detector.detect(image)
face_landmarks_list = detection_result.face_landmarks
face_landmarks = face_landmarks_list[0]

# Draw the detected landmarks
face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
face_landmarks_proto.landmark.extend([
  landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
])
landmark_list = face_landmarks_proto

# Convert image to numpy and gray
np_image = np.asarray(image.numpy_view())

# Check input is RGB image.
if np_image.shape[-1] not in (3,4):
    raise ValueError ("We expect an 3-channel RGB image for facial landmarking")
elif np_image.shape[-1] == 4:
    np_image = np_image[..., :-1]

face_image = np.copy(np_image)
gray_face_img = np.array(Image.fromarray(face_image).convert('L'))
image_rows, image_cols, _ = face_image.shape


# Build dictionary corresponding index to coordinates on face_image
idx_to_coordinates = {}
for idx, landmark in enumerate(landmark_list.landmark):
  landmark_px = normalized_to_pixel_coordinates(landmark.x, landmark.y, image_cols, image_rows)
  idx_to_coordinates[idx] = landmark_px

# Get image coordinates for eyebrows and irises
midline_right_eyebrow = get_midline_eyebrows((FACEMESH_RIGHT_EYEBROW_BOTTOM, FACEMESH_RIGHT_EYEBROW_TOP), idx_to_coordinates)
midline_left_eyebrow = get_midline_eyebrows((FACEMESH_LEFT_EYEBROW_BOTTOM, FACEMESH_LEFT_EYEBROW_TOP), idx_to_coordinates)
center_left_iris = get_center_iris(FACEMESH_LEFT_IRIS, idx_to_coordinates)
center_right_iris = get_center_iris(FACEMESH_RIGHT_IRIS, idx_to_coordinates)

### Draw eyebrows, irises, and lips on the face image

In [ ]:
# Compute the binary mask for lips
lip_mask = create_lip_mask(face_image, LOWER_LIP_INDICES + UPPER_LIP_INDICES)

# Compute the thickness for the lines that will draw the irises and eyebrows
line_thickness = compute_line_thickness(face_image, cycles_per_img=cutoff_frequency)
midline_eyebrows  = midline_right_eyebrow + midline_left_eyebrow

# Draw eyebrows on face_image
for start_coord, end_coord in midline_eyebrows:
  cv2.line(face_image, start_coord, end_coord, color=(0,0,0), thickness=line_thickness)

# Draw irises on face_image
cv2.circle(face_image, center_left_iris, radius=line_thickness, color=(0,0,0), thickness=-1)
cv2.circle(face_image, center_right_iris, radius=line_thickness, color=(0,0,0), thickness=-1)

# Set the pixels to black where the mask is True
face_image[np.all(lip_mask == 255, axis=-1)] = [0, 0, 0]

In [ ]:
plt.title('Thickened facial features overlaid')
plt.imshow(face_image)
plt.axis('off')

### Simulate a Face after preemptive landmarking

In [ ]:
gray_facial_features_overlaid_img = rgb_to_gray(img=face_image)
spatially_filtered_facial_features_overlaid_img = fft_filter(gray_facial_features_overlaid_img, face_tukey_win)
face_landmarking_img_lst = [gray_facial_features_overlaid_img, spatially_filtered_facial_features_overlaid_img]

gamma transform

In [ ]:
# @title
i = 1
plt.figure(figsize=(13, 7))
plt.suptitle('Gamma Transform')
# Loop through images and transformations
for img in face_landmarking_img_lst:
    for gamma in gamma_lst:
        ax = plt.subplot(len(face_landmarking_img_lst), len(gamma_lst), i)
        validate_gray_img(img)
        transformed_float_img = gamma_transform(img, gamma)
        ax.imshow(transformed_float_img, cmap='gray')
        ax.axis('off')
        plt.title('$\gamma =$ ' + str(gamma))
        i += 1

sigmoid transform

In [ ]:
# @title
i = 1
plt.figure(figsize=(13, 7))
plt.suptitle('Sigmoid Transform')
# Loop through images and transformations
for img in face_landmarking_img_lst:
    for gain in gain_lst:
        ax = plt.subplot(len(face_landmarking_img_lst), len(gain_lst), i)
        validate_gray_img(img)
        transformed_float_img = sigmoid_transform(img, gain, sigmoid_x_shift)
        ax.imshow(transformed_float_img, cmap='gray')
        ax.axis('off')
        plt.title('$g =$ ' + str(gain))
        i += 1

# Simulate prosthetic vision using next gen implants

cpi stands for cycles per image

In [ ]:
face_img = rgb_to_gray(image_path)
face_tukey_win = create_tukey_window(cutoff_frequency, gray_img.shape[0])
spatially_filtered_face_img = fft_filter(face_img, face_tukey_win)

In [ ]:
# define additional cutoff frequencies to test, representing potential new implants
new_cutoff_freqs_to_test = [10, 20, 50]

Inverse gamma not applied

In [ ]:
i = 1
plt.figure(figsize=(11,4))
plt.suptitle('Inverse gamma not applied')
for max_cycles_per_img in new_cutoff_freqs_to_test:
  plt.subplot(1, len(new_cutoff_freqs_to_test), i)
  quantized_face_img = quantize_image(face_img)
  new_tukey_win = create_tukey_window(max_cycles_per_img, quantized_face_img.shape[0])
  spatially_filtered = fft_filter(quantized_face_img, new_tukey_win)
  output = gamma_transform(spatially_filtered, gamma_lst[-1])
  plt.imshow(output, cmap='gray')
  plt.title('cutoff frequency ' + str(max_cycles_per_img) + ' cpi')
  plt.axis('off')
  i += 1

Inverse gamma applied

In [ ]:
i = 1
plt.figure(figsize=(11,4))
plt.suptitle('Inverse gamma applied')
for max_cycles_per_img in new_cutoff_freqs_to_test:
  plt.subplot(1, len(new_cutoff_freqs_to_test), i)
  gamma_inversed_face_img = gamma_transform(face_img, 1 / gamma_lst[-1])
  quantized_inversed_gray_img = quantize_image(gamma_inversed_face_img)
  new_tukey_win = create_tukey_window(max_cycles_per_img, quantized_inversed_gray_img.shape[0])
  spatially_filtered_gamma_inversed = fft_filter(quantized_inversed_gray_img, new_tukey_win)
  output = gamma_transform(spatially_filtered_gamma_inversed, gamma_lst[-1])
  plt.imshow(output, cmap='gray')
  plt.title('cutoff frequency ' + str(max_cycles_per_img) + ' cpi')
  plt.axis('off')
  i += 1

Inverse sigmoid not applied

In [ ]:
i = 1
plt.figure(figsize=(11,4))
plt.suptitle('Inverse sigmoid not applied')
for max_cycles_per_img in new_cutoff_freqs_to_test:
  plt.subplot(1, len(new_cutoff_freqs_to_test), i)
  quantized_face_img = quantize_image(face_img)
  new_tukey_win = create_tukey_window(max_cycles_per_img, quantized_face_img.shape[0])
  spatially_filtered = fft_filter(quantized_face_img, new_tukey_win)
  output = sigmoid_transform(spatially_filtered, gain_lst[-1], sigmoid_x_shift)
  plt.imshow(output, cmap='gray')
  plt.title('cutoff frequency ' + str(max_cycles_per_img) + ' cpi')
  plt.axis('off')
  i += 1

Inverse sigmoid applied

In [ ]:
i = 1
plt.figure(figsize=(11,4))
plt.suptitle('Inverse sigmoid applied')
for max_cycles_per_img in new_cutoff_freqs_to_test:
  plt.subplot(1, len(new_cutoff_freqs_to_test), i)
  sigmoid_inversed_face_img = inverse_sigmoid(face_img, gain_lst[-1], sigmoid_x_shift)
  quantized_inversed_face_img = quantize_image(np.clip(sigmoid_inversed_face_img, 0, 1))
  new_tukey_win = create_tukey_window(max_cycles_per_img, quantized_inversed_face_img.shape[0])
  spatially_filtered_sigmoid_inversed = fft_filter(quantized_inversed_face_img, new_tukey_win)
  output = sigmoid_transform(spatially_filtered_sigmoid_inversed, gain_lst[-1], sigmoid_x_shift)
  plt.imshow(output, cmap='gray')
  plt.title('cutoff frequency ' + str(max_cycles_per_img) + ' cpi')
  plt.axis('off')
  i += 1